## Corridas de táxi Nova York ##

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import folium
from folium import plugins
%matplotlib inline
sns.set_style('whitegrid')

** Importando arquivo e avaliando o tipo dos dados **

In [26]:
df = pd.read_json('data-sample_data-nyctaxi-trips-2009-json_corrigido.json', lines=True)

KeyboardInterrupt: 

In [ ]:
df.describe()

In [ ]:
df.head()

** Verificando e limpando dados nulos **

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(axis=1, inplace=True)

In [ ]:
df = df[df['passenger_count'] > 0]

### Qual o número total de viagens por passageiro no carro? ###

In [ ]:
df['passenger_count'].value_counts()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x='passenger_count', data=df, palette='magma')
plt.xlabel('Número de passageiros', {'fontsize': 15})
plt.ylabel('Número de viagens', {'fontsize': 15})
plt.title('Total de viagens por quantidade de passageiros', fontdict={'fontsize': 25})
plt.show()

### Qual a média de gorjeta recebida com base no número de passageiros no veículo? ###

In [ ]:
df.groupby('passenger_count')['tip_amount'].mean()

In [ ]:
plt.figure(figsize=(12,7))
plt.style.use('ggplot')
df.groupby('passenger_count')['tip_amount'].mean().plot(lw=3)
plt.ylabel('Média de gorjeta em $', fontdict={'fontsize': 15})
plt.xlabel('Número de passageiros', {'fontsize': 15})
plt.title('Média de gorjeta por número de passageiros', fontdict={'fontsize': 30})
plt.show()

### Qual a média de distância percorrida considerando todas as corridas ? ###

In [ ]:
df['trip_distance'].mean()

### Qual a média de distância percorrida com base na quantidade de passageiros no veículo ?###

In [ ]:
df.groupby('passenger_count')['trip_distance'].mean()

In [ ]:
plt.figure(figsize=(12,7))
plt.style.use('ggplot')
df.groupby('passenger_count')['trip_distance'].mean().plot(color='green', lw=3, ls='--')
plt.ylabel('Média de distância percorrida', fontdict={'fontsize': 15})
plt.xlabel('Número de passageiros', {'fontsize': 15})
plt.title('Média de distância (milhas) por número de passageiros', fontdict={'fontsize': 25})
plt.show()

** Adicionando nome das empresas ao dataframe **

In [ ]:
vendors = pd.read_csv('data-vendor_lookup-csv.csv')

In [ ]:
vendors

In [ ]:
dict_id_name = {vendors['vendor_id'][c]:vendors['name'][c] for c in range(len(vendors.vendor_id))}
dict_id_name

In [ ]:
df['Company Name'] = df['vendor_id'].map(dict_id_name)

In [ ]:
df['Company Name'].head()

### Quais as maiores empresas com base no total de dinheiro arrecadado ? ###

In [ ]:
df.groupby('Company Name')['total_amount'].sum()

In [ ]:
plt.figure(figsize=(12,7))
plt.style.use('ggplot')
sns.barplot(x='Company Name', y='total_amount', palette='bright', data=df, estimator=np.sum)
plt.ylabel('Total em dólar $', fontdict={'fontsize': 15})
plt.xlabel('Nome da empresa', {'fontsize': 15})
plt.title('Total em dólar arrecadado por empresa', fontdict={'fontsize': 30})
plt.show()

** Tipos de pagamento **

In [ ]:
df['payment_type'].unique()

In [ ]:
df['payment_type'] = df['payment_type'].apply(lambda value: value.lower())

### Qual o total de dinheiro arrecadado com base no tipo de pagemento? ##

In [ ]:
df.groupby('payment_type')['total_amount'].sum()

In [ ]:
sns.catplot(x='payment_type', y='total_amount', data=df, estimator=np.sum, height=7, aspect=1.4, kind='point', color='purple')
plt.xlabel('Tipo de pagamento', fontdict={'fontsize': 15})
plt.ylabel('Total em dólar $', fontdict={'fontsize': 15})
plt.title('Total em dólar por tipo de pagamento', fontdict={'fontsize': 30})
plt.show()

** Adicionando colunas de tempo ao dataframe **

In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

In [ ]:
months = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October',
         11:'November', 12:'December'}

In [ ]:
df['Day'] = df['pickup_datetime'].apply(lambda time: time.day)
df['Month'] = df['pickup_datetime'].apply(lambda time: time.month)
df['Month'] = df['Month'].map(months)

In [ ]:
df['Hour Pickup'] = df['pickup_datetime'].apply(lambda time: time.time)
df['Hour Dropoff'] = df['dropoff_datetime'].apply(lambda time: time.time)

In [ ]:
df['Hour'] = df['Hour Pickup'].apply(lambda time: time.hour)

In [ ]:
df.head()

### Mapa de calor - quantidade total de passageiros - dia x hora ###

In [ ]:
day_month = df.pivot_table(values='passenger_count', index='Hour', columns='Day', aggfunc=np.sum, fill_value=0)
day_month

In [ ]:
plt.figure(figsize=(14,9))
sns.heatmap(day_month,cmap='coolwarm',linecolor='white',linewidths=1)
plt.ylabel('Horas', fontdict={'fontsize': 15})
plt.xlabel('Dias do mês', {'fontsize': 15})
plt.title('Total de passageiros', fontdict={'fontsize': 25})
plt.show()

### Correlação entre as colunas do dataframe ###

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(df.corr(),cmap='coolwarm',annot=True)
plt.title('Correlação entre dados', fontdict={'fontsize': 20}, weight='light')
plt.tight_layout()
plt.show()

### Mapa de calor - destino inicial ###

In [27]:
coordenadas = []
lat = df['pickup_latitude'][::2500].values
long = df['pickup_longitude'][::2500].values

for la,lo in zip(lat,long):
    coordenadas.append([la,lo])

mapa = folium.Map(location=[40.6643,-73.9385],tiles='Stamen Toner',zoom_start=10)
mapa.add_child(plugins.HeatMap(coordenadas))
mapa

### Mapa de calor - destino final ###

In [28]:
coordenadas1 = []
lat = df['dropoff_latitude'][::2500].values
long = df['dropoff_longitude'][::2500].values

for la,lo in zip(lat,long):
    coordenadas1.append([la,lo])

mapa = folium.Map(location=[40.6643,-73.9385],tiles='Stamen Toner',zoom_start=10)
mapa.add_child(plugins.HeatMap(coordenadas1))
mapa